<a href="https://colab.research.google.com/github/showna-wei/ForColabUse/blob/main/google_genai/content_api_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2025 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-gemini/genai-processors/blob/main/notebooks/content_api_intro.ipynb)

# Welcome to the GenAI Processors Content API

This notebook is your friendly introduction to the `content_api` module within
the GenAI Processors library. We'll explore how to create, manipulate, and
interact with the fundamental building blocks of content: `ProcessorPart` and
`ProcessorContent`.

**What you'll learn:**

*   How to create `ProcessorPart` objects from various data types (strings,
    bytes, PIL Images, GenAI Parts).
*   The key attributes of a `ProcessorPart` (like `mimetype`, `substream_name`,
    `role`, `metadata`).
*   How to construct `ProcessorContent` objects to group multiple
    `ProcessorPart`s.
*   Useful utility functions for working with content (e.g., `as_text`).
*   How `ProcessorPart` and `ProcessorContent` integrate with GenAI models.

Let's dive in and unlock the power of structured content in your AI pipelines!

## 1. 🛠️ Setup

In [ ]:
!pip install genai_processors

In [ ]:
# @title Import modules
import dataclasses
import dataclasses_json
from genai_processors import content_api
from genai_processors import processor
from genai_processors import streams
from genai_processors.core import genai_model
from google.colab import userdata
from google.genai import types as genai_types
from IPython.display import display
import nest_asyncio
import PIL.Image
import requests

nest_asyncio.apply()  # Needed to run async loops in Colab

# Convenient aliases
ProcessorPart = content_api.ProcessorPart
ProcessorContent = content_api.ProcessorContent
as_text = content_api.as_text
is_text = content_api.is_text
is_json = content_api.is_json

# For GenAI Model interaction (optional, but useful for demonstration!)
try:
  API_KEY = userdata.get("GOOGLE_API_KEY")
  if not API_KEY:
    print(
        "⚠️ API Key not found in Colab secrets. GenAI model examples will be"
        " skipped."
    )
    print(
        "To run these, add your Gemini API Key to Colab Secrets with the name"
        " 'GOOGLE_API_KEY'."
    )
except userdata.SecretNotFoundError:
  API_KEY = None  # Or set your API key directly here if not using Colab secrets
  print(
      "`userdata` not imported. Set API_KEY manually if you want to run GenAI"
      " model examples."
  )

GenaiModel = genai_model.GenaiModel


def generate_gdm_logo_bytes() -> bytes:
  # The URL of the DeepMind logo image
  image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/DeepMind_new_logo.svg/2560px-DeepMind_new_logo.svg.png"
  headers = {"User-Agent": "genai_processors Colab"}
  response = requests.get(image_url, headers=headers)
  response.raise_for_status()
  return response.content

## 2. 🧱 ProcessorPart: The Atomic Unit of Content

A `ProcessorPart` is the smallest, indivisible piece of content your processors
will handle. Think of it as a typed container for a single modality and a single
role.

You can create `ProcessorPart` objects from various Python types:

In [ ]:
text_data = "Hello, GenAI World! This is a text part."
text_part = ProcessorPart(text_data)

print(f"Original data: {text_data}")
print(f"ProcessorPart: {text_part}")
print(
    f"MIME type: {text_part.mimetype}"
)  # Automatically inferred as 'text/plain'
print(f"Text content: '{text_part.text}'")

#### Key Attributes of ProcessorPart:

*   `part`: The underlying `google.genai.types.Part` object.
*   `text`: Access the text content (raises ValueError if not text).
*   `bytes`: Access the content as bytes.
*   `pil_image`: Access the content as a PIL Image (raises ValueError if not an
    image).
*   `mimetype`: The
    [MIME type](https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/MIME_types)
    of the content (e.g., `text/plain`, `image/png`, `application/json`).
*   `role`: Indicates the producer of the content (e.g., `"user"`, `"model"`,
    `"tool"`). Useful for conversational AI. Default is `""`.
*   `substream_name`: A custom string to categorize or route parts. Useful for
    distinguishing different types of information or alternative responses.
    Default is `""`.
    -   Some substream names have special meaning in processors; for example,
        `status` and `debug` are reserved for content that needs to be returned
        early to the user and will not be processed further down the pipeline.
    -   `realtime` is for content that will use the Live API
        `send_realtime_content()` method (in contrast to
        `send_client_content()`).
*   `metadata`: A dictionary for any other arbitrary information you want to
    attach to the part.

Let's create a text part with more attributes:

In [ ]:
detailed_text_part = ProcessorPart(
    "This is a user query.",
    role="user",
    substream_name="user_query_main",
    metadata={
        "timestamp": "2024-07-29T10:00:00Z",
        "session_id": "xyz123",
    },
)

print(f"ProcessorPart: {detailed_text_part}")
print(f"Role: {detailed_text_part.role}")
print(f"Substream Name: {detailed_text_part.substream_name}")
print(f"Custom Metadata: {detailed_text_part.metadata}")
print(
    f"Timestamp from metadata: {detailed_text_part.get_metadata('timestamp')}"
)

### From Bytes (e.g., Image Data)

When creating a `ProcessorPart` from raw bytes, you **must** specify the
`mimetype`.

In [ ]:
gdm_png_bytes = generate_gdm_logo_bytes()

image_bytes_part = ProcessorPart(gdm_png_bytes, mimetype="image/png")

print(f"ProcessorPart (from bytes): {image_bytes_part}")
print(f"MIME type: {image_bytes_part.mimetype}")
print(f"Has bytes: {image_bytes_part.bytes is not None}")

# You can access it as a PIL Image too
try:
  pil_img = image_bytes_part.pil_image
  display(pil_img)
except Exception as e:
  print(f"Error converting to PIL Image: {e}")

### From a PIL (Pillow) Image Object

You can directly pass a `PIL.Image.Image` object. The library will handle
converting it to bytes and inferring the MIME type (defaults to `image/webp` if
the PIL Image has no format, or uses its existing format like `image/png`,
`image/jpeg`).

In [ ]:
# Create a simple PIL Image
pil_image_obj = PIL.Image.new("RGB", (60, 30), color="red")
pil_image_part = ProcessorPart(pil_image_obj)

print(f"ProcessorPart (from PIL Image): {pil_image_part}")
print(
    f"Inferred MIME type: {pil_image_part.mimetype}"
)  # Likely image/webp or image/png
display(pil_image_part.pil_image)  # Display in Colab

# You can also specify a mimetype if you want a different format
jpeg_pil_image_part = ProcessorPart(pil_image_obj, mimetype="image/jpeg")
print(f"ProcessorPart (from PIL with specified JPEG): {jpeg_pil_image_part}")
print(f"MIME type: {jpeg_pil_image_part.mimetype}")
display(jpeg_pil_image_part.pil_image)

### From a `google.genai.types.Part`

If you're working with the Google AI SDK, you can directly wrap
`genai_types.Part` objects in a `ProcessorPart`.

In [ ]:
genai_text_part_sdk = genai_types.Part(text="From GenAI SDK!")
processor_part_from_sdk = ProcessorPart(genai_text_part_sdk, role="model")

print(f"ProcessorPart (from GenAI SDK Part): {processor_part_from_sdk}")
print(f"Text: {processor_part_from_sdk.text}")
print(f"Role: {processor_part_from_sdk.role}")

### From another ProcessorPart

Creating a `ProcessorPart` from an existing `ProcessorPart` creates a new
instance, which uses the underlying `genai_types.Part` of the existing
`ProcessorPart`. This means changes to the underlying `genai_types.Part` in the
original `ProcessorPart` will be reflected in the new one.

You can override attributes like `role`, `substream_name`, and `metadata` when
creating a new `ProcessorPart` from an existing one.

In [ ]:
original_part = ProcessorPart(
    "Original message.",
    role="user",
    substream_name="original_stream",
    metadata={"version": 1},
)

# Simple copy
copied_part = ProcessorPart(original_part)
print(f"Original: {original_part}")
print(f"Copied:   {copied_part}")
print(f"Are they the same object? {original_part is copied_part}")  # False
print(f"Are they equal in value? {original_part == copied_part}")  # True

# Copy with overridden attributes
modified_copy_part = ProcessorPart(
    original_part,
    role="model",  # Changed role
    substream_name="modified_stream",  # Changed substream
    metadata={"version": 2, "status": "processed"},  # New metadata
)
print(f"\nModified Copy: {modified_copy_part}")
print(f"Role: {modified_copy_part.role}")
print(f"Substream: {modified_copy_part.substream_name}")
print(f"Metadata: {modified_copy_part.metadata}")

### From a Python Dataclass (Structured Data)

For structured data, you can use `ProcessorPart.from_dataclass()`. This will
serialize your dataclass to JSON and set the `mimetype` to `application/json;
type=<ClassName>`.

In [ ]:
@dataclasses_json.dataclass_json  # Important for JSON serialization
@dataclasses.dataclass
class MyStructuredData:
  id: int
  name: str
  tags: list[str]


my_data_instance = MyStructuredData(
    id=101, name="Alpha", tags=["important", "beta"]
)

dataclass_part = ProcessorPart.from_dataclass(
    dataclass=my_data_instance,
    role="system_event",
    metadata={"source": "internal_module"},
)

print(f"ProcessorPart (from dataclass): {dataclass_part}")
print(f"MIME type: {dataclass_part.mimetype}")
print(f"Underlying text (JSON): {dataclass_part.text}")
assert is_json(dataclass_part.mimetype)

# To get the dataclass back:
retrieved_data_instance = dataclass_part.get_dataclass(MyStructuredData)
print(f"\nRetrieved dataclass instance: {retrieved_data_instance}")
print(
    "Is original equal to retrieved?"
    f" {my_data_instance == retrieved_data_instance}"
)

## 3. 📦 ProcessorContent: A Collection of Parts

`ProcessorContent` is a container for one or more `ProcessorPart` objects. It is
often used to represent a complete turn in a conversation or a collection of
multimodal inputs.

It behaves like a list of `ProcessorPart`s and can be constructed by passing
`ProcessorPart` instances (or data that can be converted to `ProcessorPart`s) to
its constructor.

### Creating ProcessorContent

In [ ]:
# From individual strings/parts
content1 = ProcessorContent(
    "This is the first part.",
    ProcessorPart(generate_gdm_logo_bytes(), mimetype="image/png", role="user"),
    "And a final textual comment.",
)

print("Content 1:")
for part in content1:  # You can iterate directly over ProcessorContent
  print(
      f"  - {part.mimetype}:"
      f" {part.text if is_text(part.mimetype) else '[binary data]'}"
  )
print(f"Length of Content 1: {len(content1)}")

# From a list of ProcessorPart objects
parts_list = [
    ProcessorPart("Query about cats.", role="user"),
    ProcessorPart("Cats are fascinating creatures!", role="model"),
]
content2 = ProcessorContent(parts_list)

print("\nContent 2:")
for (
    mime,
    part_obj,
) in content2.items():  # .items() yields (mimetype, ProcessorPart)
  print(f"  - Role: {part_obj.role}, Mimetype: {mime}, Text: {part_obj.text}")

# From another ProcessorContent object (creates a new collection)
content3 = ProcessorContent(content1)
print(f"\nContent 3 (copy of Content 1):")
print(f"Is Content 1 same object as Content 3? {content1 is content3}")
print(f"Is Content 1 equal to Content 3? {content1 == content3}")

### Utility: `as_text()`

A common task is to extract all textual information from a `ProcessorContent`
object. The `content_api.as_text()` function does exactly this, concatenating
the text from all text-based parts. You can specify the `substream_name`
argument to extract the text of a given substream.

In [ ]:
multimodal_content = ProcessorContent(
    ProcessorPart("Here is some initial text. ", substream_name="other"),
    ProcessorPart(
        generate_gdm_logo_bytes(), mimetype="image/png"
    ),  # This will be ignored by as_text
    "Followed by more text. ",
)

all_text = as_text(multimodal_content)
print(f"Concatenated text from multimodal_content:\n{all_text}")

other_text_only = as_text(multimodal_content, substream_name="other")
print(f"Text from 'other' substream:\n{other_text_only}")

## 4. 🤖 Integration with GenAI Models (Optional)

The `ProcessorPart` and `ProcessorContent` are designed to work seamlessly with
GenAI models via the `GenaiModel` processor. The `GenaiModel` processor expects
an `AsyncIterable[ProcessorPart]` as input, and `ProcessorContent` (or a list of
`ProcessorPart`s, or even strings) can be easily converted to such a stream
using `streams.stream_content()`.

#### Async

In [ ]:
p_genai = GenaiModel(
    api_key=API_KEY,
    model_name="gemini-2.0-flash-lite",
)
input_stream = streams.stream_content(
    "What is the best part of owning a Dalmatian?"
)
async for content_part in p_genai(input_stream):
  print(content_part.text)

#### Sync

If you are in a sync environment and prefer to block on the model call, use the
`apply_sync` method, which takes a list of `ProcessorPart` objects as input.

In [ ]:
p = GenaiModel(
    api_key=API_KEY,
    model_name='gemini-2.0-flash-lite',
)

genai_content = processor.apply_sync(
    p, ['What is the best part of owning a Dalmatian?']
)

print('Raw parts\n\n')
for content_part in genai_content:
  print(content_part)
print('\n\n')
print('Using `content_api.as_text:\n\n')
print(content_api.as_text(genai_content))

## 5. Next Steps

This tutorial covered the creation of `ProcessorPart` and `ProcessorContent`
objects and how they form the basis of content handling in GenAI Processors. You
also saw how they integrate with GenAI models.

Check the
[processor intro](https://colab.research.google.com/github/google-gemini/genai-processors/blob/main/notebooks/processor_intro.ipynb)
notebook to dive deeper into creating real-time processors using the Live API.